Author: Aditya

In [1]:
import pandas as pd
import numpy as np

In [2]:
plans_df = pd.read_excel("key_top10k_links.xlsx", header=0, sheet_name="unique-linksearch")

In [3]:
plans_df.head()

,index,BIN_top10k,PCN_top10k,GRP_top10k,PLAN_NAME,ROW_CNT,RATIO,BPG_top10k,COMMENTS,PDF_LINKS,Assigned_to
0,154,17423,IRX,AMERISAV,NaN,18004261,0.000725,017423IRXAMERISAV,not found,NaN,Aditya
1,157,1553,GAM,NaN,MEDICAID GEORGIA,17778171,0.000716,001553GAM(NULL),NaN,https://professionals.optumrx.com/content/dam/...,Aditya
2,161,4336,ADV,RX1021,PCS ADVANCE PARADIGM,17514092,0.000706,004336ADVRX1021,NaN,https://www.caremark.com/portal/asset/D0PayerS...,Aditya
3,162,610502,00670000,RX7704,AETNA,17450172,0.000703,61050200670000RX7704,NaN,https://www.caremark.com/portal/asset/D0PayerS...,Aditya
4,163,3858,A4,PMDM,EXPRESS SCRIPTS,17282267,0.000696,003858A4PMDM,NaN,https://www.express-scripts.com/art/prc/NCPDP_...,Aditya


In [4]:
plans_df = plans_df[(plans_df["Assigned_to"] == "Aditya") & (plans_df["PDF_LINKS"].isna()) & (plans_df["COMMENTS"] != "not found")]

In [5]:
plans_df.shape

(262, 11)

In [6]:
plans_df.iloc[0]

index                            226
BIN_top10k                    610144
PCN_top10k                   ALBCASH
GRP_top10k                       NaN
PLAN_NAME      CASH PLAN  ALBERTSONS
ROW_CNT                     13841956
RATIO                       0.000558
BPG_top10k       610144ALBCASH(NULL)
COMMENTS                         NaN
PDF_LINKS                        NaN
Assigned_to                   Aditya
Name: 29, dtype: object

In [7]:
plans_df.head()

,index,BIN_top10k,PCN_top10k,GRP_top10k,PLAN_NAME,ROW_CNT,RATIO,BPG_top10k,COMMENTS,PDF_LINKS,Assigned_to
29,226,610144,ALBCASH,NaN,CASH PLAN ALBERTSONS,13841956,0.000558,610144ALBCASH(NULL),NaN,NaN,Aditya
31,231,9737,P008009737,NaN,MEDICAID MICHIGAN,13579829,0.000547,009737P008009737(NULL),NaN,NaN,Aditya
34,239,610674,ABARCAD,NaN,MEDICARE PART D ABARCA HEALTH ABARCA D,13108508,0.000528,610674ABARCAD(NULL),NaN,NaN,Aditya
37,264,17423,IRX,10042,NaN,12277494,0.000495,017423IRX10042,NaN,NaN,Aditya
41,277,4682,CN,NaN,THERAPY FIRST,11803747,0.000476,004682CN(NULL),NaN,NaN,Aditya


In [8]:
plans_df.dtypes

index            int64
BIN_top10k       int64
PCN_top10k      object
GRP_top10k      object
PLAN_NAME       object
ROW_CNT          int64
RATIO          float64
BPG_top10k      object
COMMENTS        object
PDF_LINKS       object
Assigned_to     object
dtype: object

In [9]:
plans_df_test = plans_df.iloc[0:10]

In [10]:
plans_df_rem = plans_df_test[["BIN_top10k", "PCN_top10k", "GRP_top10k"]]

In [11]:
plans_df_rem

,BIN_top10k,PCN_top10k,GRP_top10k
29,610144,ALBCASH,NaN
31,9737,P008009737,NaN
34,610674,ABARCAD,NaN
37,17423,IRX,10042
41,4682,CN,NaN
42,610602,MCD,NaN
43,4766,USMCA,NaN
46,5526,MEPOP,NaN
47,15599,05440000,NaN
48,17010,0215COMM,NaN


In [12]:
import os
import requests
from urllib.parse import urlparse
# from langchain_ollama import OllamaLLM
# from langchain.prompts import PromptTemplate
from ddgs import DDGS

In [13]:
# Create folders
PDF_FOLDER = "output/pdfs"
EXCEL_FOLDER = "output/excels"
os.makedirs(PDF_FOLDER, exist_ok=True)
os.makedirs(EXCEL_FOLDER, exist_ok=True)


# Initialize Ollama LLM (make sure Ollama is running!)
# llm = OllamaLLM(model="phi3")

# Helper: classify file type
def get_file_type(url):
    url = url.lower()
    if ".pdf" in url:
        return "pdf"
    if any(ext in url for ext in [".xls", ".xlsx", ".csv"]):
        return "excel"
    return None

# Helper: download file
def download_file(url, save_dir, prefix):
    try:
        response = requests.get(url, timeout=10)
        filename = os.path.basename(urlparse(url).path)

        # Add prefix (BIN-PCN-GRP) to filename to track its origin
        filename = f"{prefix}_{filename}"
        filepath = os.path.join(save_dir, filename)

        with open(filepath, "wb") as f:
            f.write(response.content)
        print(f"Downloaded: {url}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")


for idx, row in plans_df_test.iterrows():
    # Get fields, use "NA" if PCN or GRP is missing
    bin_ = str(row['BIN_top10k']).strip()
    pcn = str(row['PCN_top10k']).strip() if pd.notna(row['PCN_top10k']) else "NA"
    grp = str(row['GRP_top10k']).strip() if pd.notna(row['GRP_top10k']) else "NA"

    # For folder and filename consistency
    plan_id = f"{bin_}_{pcn}_{grp}"

    # Build the base query dynamically
    query_parts = [f"Bin {bin_}"]
    if pcn != "NA":
        query_parts.append(f"PCN {pcn}")
    if grp != "NA":
        query_parts.append(f"GroupID {grp}")
    base_query = " ".join(query_parts) + " payer sheet"

    print(f"\nProcessing: {plan_id}")
    print(f"Base query: {base_query}")

    # Ask LLM for refined queries
    # prompt = PromptTemplate.from_template(
    #     "Suggest 3 good search queries to find PDF or Excel payer sheet documents for this insurance plan:\n\nQuery: {query}"
    # )
    # response = llm.invoke(prompt.format(query=base_query))
    # refined_queries = [q.strip() for q in response.strip().split("\n") if q.strip()]
    queries = [base_query] 
    # + refined_queries

    # Create plan-specific folders
    plan_pdf_folder = os.path.join(PDF_FOLDER, plan_id)
    plan_excel_folder = os.path.join(EXCEL_FOLDER, plan_id)
    os.makedirs(plan_pdf_folder, exist_ok=True)
    os.makedirs(plan_excel_folder, exist_ok=True)

    # Search and download
    with DDGS() as ddgs:
        found = False
        for q in queries:
            print(f"Trying: {q}")
            results = ddgs.text(q, max_results=10)
            for r in results:
                url = r.get("href") or r.get("url")
                if not url:
                    continue
                ftype = get_file_type(url)
                if ftype == "pdf":
                    download_file(url, plan_pdf_folder, plan_id)
                    found = True
                elif ftype == "excel":
                    download_file(url, plan_excel_folder, plan_id)
                    found = True
            if found:
                break
        if not found:
            print("No files found.")


Processing: 610144_ALBCASH_NA
Base query: Bin 610144 PCN ALBCASH payer sheet
Trying: Bin 610144 PCN ALBCASH payer sheet
Downloaded: https://www.caremark.com/portal/asset/D0PayerSheetMEDDMSP.pdf
Downloaded: https://www.aetna.com/document-library/pharmacy-insurance/healthcare-professional/documents/aetna-payer-sheet-commercial-2019.pdf
Downloaded: https://corp-strapi.ascellahealth.com/uploads/Claim_Billing_and_Response_Payer_Sheet_BIN_s_017290_018448_019505_020503_020511_021957_effective_07_05_2023_4301d61186_916c3c4612.pdf

Processing: 9737_P008009737_NA
Base query: Bin 9737 PCN P008009737 payer sheet
Trying: Bin 9737 PCN P008009737 payer sheet
Downloaded: https://mi.primetherapeutics.com/provider/external/medicaid/mi/doc/en-us/MIRx_D0_claims_processing_manual.pdf

Processing: 610674_ABARCAD_NA
Base query: Bin 610674 PCN ABARCAD payer sheet
Trying: Bin 610674 PCN ABARCAD payer sheet
Downloaded: https://abarcahealth.com/wp-content/uploads/2023/06/2023-Provider-Manual.pdf
Downloaded: htt